In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime, timedelta

# Load functions to load data and make decision trees
%run ./load_nc_and_subset.ipynb

In [10]:
# Import data


sesrFName = 'sesr_all_years_USDMTimeScale_conus.nc'
spiFName  = 'SPI_all_years_USDMTimeScale_conus.nc'
usdmFName = 'USDM_grid_all_years.nc'

sesrSName = 'sesr'
spiSName  = 'SPI'
usdmSName = 'USDM'

sesr = LoadNC(sesrFName, sesrSName)
spi  = LoadNC(spiFName, spiSName)
usdm = LoadNCnomask(usdmFName, usdmSName)

FileNotFoundError: [Errno 2] No such file or directory: b'sesrsesr_all_years_USDMTimeScale_conus.nc'

In [ ]:
# Process data

usdm['USDM'][usdm['USDM'] > 0] = 1

TestInd = np.where( (usdm['year'] == 2011) | (usdm['year'] == 2019) )[0]
TrainValInd = np.where( (usdm['year'] != 2011) & (usdm['year'] != 2019) )[0]

sesr_test = sesr['sesr'][:,:,TestInd]
spi_test  = spi['SPI'][:,:,TestInd]
usdm_test = usdm['USDM'][:,:,TestInd]

sesr_TrainVal = sesr['sesr'][:,:,TrainValInd]
spi_TrainVal  = spi['SPI'][:,:,TrainValInd]
usdm_TrainVal = usdm['USDM'][:,:,TrainValInd]

ValInd = np.where(usdm['year'] == 2017)[0]
TrainInd = np.where( (usdm['year'] != 2011) & (usdm['year'] != 2017) & (usdm['year'] != 2019) )[0]

sesr_train = sesr['sesr'][:,:,TrainInd]
spi_train  = spi['SPI'][:,:,TrainInd]
usdm_train = usdm['USDM'][:,:,TrainInd]

sesr_val = sesr['sesr'][:,:,ValInd]
spi_val  = spi['SPI'][:,:,ValInd]
usdm_val = usdm['USDM'][:,:,ValInd]

# Collect the training and validation datasets. Use 2017, drought in northern plains, as a validation set
ValInd = np.where(usdm['year'] == 2017)[0]
TrainInd = np.where( (usdm['year'] != 2011) & (usdm['year'] != 2017) & (usdm['year'] != 2019) )[0]

sesr_train = sesr['sesr'][:,:,TrainInd]
spi_train  = spi['SPI'][:,:,TrainInd]
usdm_train = usdm['USDM'][:,:,TrainInd]

sesr_val = sesr['sesr'][:,:,ValInd]
spi_val  = spi['SPI'][:,:,ValInd]
usdm_val = usdm['USDM'][:,:,ValInd]

# Transform the data into 1D arrays for easier iterations in the SL learning.
I, J, T_train = usdm_train.shape
T_val         = usdm_val.shape[-1]
T_train_val   = usdm_TrainVal.shape[-1]
T_test        = usdm_test.shape[-1]

# USDM is the label, therefore is the y vector in the learning algorithms.
y_train     = usdm_train.reshape(I*J*T_train, order = 'F')
y_val       = usdm_val.reshape(I*J*T_val, order = 'F')
y_train_val = usdm_TrainVal.reshape(I*J*T_train_val, order = 'F')
y_test      = usdm_test.reshape(I*J*T_test, order = 'F')


sesr_train1D     = sesr_train.reshape(I*J*T_train, order = 'F')
sesr_val1D       = sesr_val.reshape(I*J*T_val, order = 'F')
sesr_train_val1D = sesr_TrainVal.reshape(I*J*T_train_val, order = 'F')
sesr_test1D      = sesr_test.reshape(I*J*T_test, order = 'F')

spi_train1D     = spi_train.reshape(I*J*T_train, order = 'F')
spi_val1D       = spi_val.reshape(I*J*T_val, order = 'F')
spi_train_val1D = spi_TrainVal.reshape(I*J*T_train_val, order = 'F')
spi_test1D      = spi_test.reshape(I*J*T_test, order = 'F')


# Finally, the features are the compination of SESR and SPI
# For x, the first column is SESR, the second is SPI. shape[-1] is the number of features
x_train     = np.asarray([sesr_train1D, spi_train1D]).T
x_val       = np.asarray([sesr_val1D, spi_val1D]).T
x_train_val = np.asarray([sesr_train_val1D, spi_train_val1D]).T
x_test      = np.asarray([sesr_test1D, spi_test1D]).T

# Quick note for the case of perceptrons, any type of regression, etc.
# A column for the bias needs to be included, so a few more lines are needed.
# e.g., for the training data set, the following line would be needed:
# ones_train = np.ones((T_train))
# x_train = np.asarray([ones_train, sesr_train1D, spi_train1D]).T
# and so on for the other datasets.

In [ ]:
# Create Perceptron class for predictions
class Perceptron:
  
  # Constructor for when a Perceptron is created
  def __init__ (self):
    self.w = None
    self.b = None
    
  # Model 
  def model(self, x):
        
    return 1 if (np.dot(self.w, x) >= self.b) else 0
  
  #predictor to predict on the data based on w
  def predict(self, X):
    Y = []
    for x in X:
      result = self.model(x)
      Y.append(result)
    return np.array(Y)
    
  def fit(self, X, Y, epochs = 1, lr = 1):
    self.w = np.ones(X.shape[1])
    self.b = 0
    accuracy = {}
    max_accuracy = 0
    wt_matrix = []
    
    # Update the weight array epochs times for each element in the training arrays
    for i in range(epochs):
      for x, y in zip(X, Y):
        y_pred = self.model(x)
        if y == 1 and y_pred == 0:
          self.w = self.w + lr * x
          self.b = self.b - lr * 1
        elif y == 0 and y_pred == 1:
          self.w = self.w - lr * x
          self.b = self.b + lr * 1
          
      wt_matrix.append(self.w)    
      accuracy[i] = accuracy_score(self.predict(X), Y)
      if (accuracy[i] > max_accuracy):
        max_accuracy = accuracy[i]
        chkptw = self.w
        chkptb = self.b
        
    #checkpoint (Save the weights and b value)
    self.w = chkptw
    self.b = chkptb
    
    #return the weight matrix, that contains weights over all epochs
    return np.array(wt_matrix)

In [ ]:
# Run the created Perceptron
perc = Perceptron()

# Fitting Training Data with 100 epochs and a learning rate of .5
# This test tries to see if SESR has a link to USDM

w_matrix = perc.fit(x_train[0], y_train, 100, .5)

y_val_predict = prec.predict(x_val[0])

print(y_val_predict)